<a href="https://colab.research.google.com/github/DolevSeren/DI-Bootcamp/blob/main/Fine_Tuning_GPT_2_For_SMS_Spam.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%pip install --quiet datasets evaluate transformers[sentencepiece]


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 4.7 MB/s eta 0:00:00


In [2]:
import pandas as pd

df = pd.read_parquet("hf://datasets/ucirvine/sms_spam/plain_text/train-00000-of-00001.parquet")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [3]:
from datasets import Dataset

# נניח שהדאטה שלך נראה כך:
# df.columns => ['sms', 'label']

# דגימה אקראית: 4000 שורות ל־train, 1000 ל־validation
train_df = df.sample(n=4000, random_state=42)
val_df = df.drop(train_df.index).sample(n=1000, random_state=42)

# המרה ל־Dataset
train_ds = Dataset.from_pandas(train_df)
val_ds = Dataset.from_pandas(val_df)

# בדיקה
print(train_ds.features)


{'sms': Value(dtype='string', id=None), 'label': Value(dtype='int64', id=None), '__index_level_0__': Value(dtype='int64', id=None)}


In [4]:
from transformers import GPT2Tokenizer

# נשתמש ב-gpt2
model_name = "gpt2"

# טוענים את הטוקניזר של GPT2
tokenizer = GPT2Tokenizer.from_pretrained(model_name)

# מגדירים pad_token להיות כמו ה-end-of-sequence token
tokenizer.pad_token = tokenizer.eos_token

# פונקציית טוקניזציה שמתאימה לשימוש עם Hugging Face Datasets
def tokenize_fn(examples):
    return tokenizer(
        examples["sms"],              # השדה בדאטהסט שמכיל את ההודעה
        padding="max_length",         # מוסיף padding כדי שהרצפים יהיו באורך אחיד
        truncation=True,              # קוצץ טקסטים ארוכים
        max_length=64                 # אורך מקסימלי של רצף (מספיק ל-SMS)
    )

# מיישמים את הפונקציה על כל הדאטהסט
train_tok = train_ds.map(tokenize_fn, batched=True)
val_tok = val_ds.map(tokenize_fn, batched=True)


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

Map:   0%|          | 0/4000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [5]:
import torch
from transformers import GPT2ForSequenceClassification

# טוענים את המודל GPT-2 עם ראש לסיווג
model = GPT2ForSequenceClassification.from_pretrained(
    model_name,         # "gpt2"
    num_labels=2,       # שתי תוויות: spam (1) או ham (0)
    pad_token_id=tokenizer.eos_token_id  # יש צורך להגדיר את ה-pad token
)


model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [6]:
import evaluate
import numpy as np

# טוענים את מדדי ההערכה
accuracy  = evaluate.load("accuracy")
precision = evaluate.load("precision")
recall    = evaluate.load("recall")
f1        = evaluate.load("f1")

# פונקציה שמחשבת את כל המדדים
def compute_metrics(pred):
    logits, labels = pred
    preds = np.argmax(logits, axis=-1)
    return {
        "accuracy":  accuracy.compute(predictions=preds, references=labels)["accuracy"],
        "precision": precision.compute(predictions=preds, references=labels)["precision"],
        "recall":    recall.compute(predictions=preds, references=labels)["recall"],
        "f1":        f1.compute(predictions=preds, references=labels)["f1"]
    }


In [7]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="./results",            # לאן לשמור את המודל המאומן
    do_train=True,                     # לבצע אימון
    do_eval=True,                      # לבצע הערכה            # תיקיית לוגים
    logging_steps=500,                 # תדירות לוגים

    per_device_train_batch_size=8,     # באטץ' לאימון
    per_device_eval_batch_size=8,      # באטץ' להערכה
    num_train_epochs=1,                # מספר אפוקים
    learning_rate=5e-5,                # קצב למידה
    weight_decay=0.01,                 # רגולריזציה כדי למנוע overfitting

    report_to=None,                    # בלי דיווח ל-wandb וכדומה
    save_total_limit=1,                # רק checkpoint אחד יישמר
)


In [8]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_tok,
    eval_dataset=val_tok,
    compute_metrics=compute_metrics
)

# אימון
trainer.train()

# הערכה
metrics = trainer.evaluate()
print(metrics)


wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: dolevseren (dolevseren-developers-institute) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Step,Training Loss
500,0.103400


{'eval_loss': 0.08468008786439896, 'eval_accuracy': 0.986, 'eval_precision': 0.9632352941176471, 'eval_recall': 0.9357142857142857, 'eval_f1': 0.9492753623188406, 'eval_runtime': 276.8845, 'eval_samples_per_second': 3.612, 'eval_steps_per_second': 0.451, 'epoch': 1.0}
